# Transfer Learning

### Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image
import json
from tqdm import tqdm

import os
os.chdir("C:/Users/victor/Documents/ECOLE/5A/T1/NEURONES/TP/MiniProjet/GTSDB/")


### Classe GTSBD 

In [2]:
class GTSDBDataset(Dataset):
    def __init__(self, json_file, transform=None, label_to_idx=None):
        with open(json_file, "r") as f:
            self.samples = json.load(f)
        self.transform = transform
        self.label_to_idx = label_to_idx

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        label = self.label_to_idx[label]

        return img, label


### Transformations de nos images

In [3]:
efficientnet_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


### Chargement du JSON et construction des labels 

In [6]:
with open("data/64x64/train.json", "r") as f:
    train_samples = json.load(f)


labels = [s[1] for s in train_samples]
unique_labels = sorted(list(set(labels)))

label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}
idx_to_label = {idx: label for label, idx in label_to_idx.items()}

label_to_idx


{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 7: 5,
 8: 6,
 9: 7,
 10: 8,
 11: 9,
 12: 10,
 13: 11,
 14: 12,
 17: 13,
 18: 14,
 25: 15,
 38: 16}

### Chargement datatset avec mapping des labels

In [7]:
train_data = GTSDBDataset("data/64x64/train.json", efficientnet_transform, label_to_idx)
val_data   = GTSDBDataset("data/64x64/val.json",   efficientnet_transform, label_to_idx)
test_data  = GTSDBDataset("data/64x64/test.json",  efficientnet_transform, label_to_idx)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_data,   batch_size=32, shuffle=False)
test_loader  = DataLoader(test_data,  batch_size=32, shuffle=False)

num_classes = len(label_to_idx)
num_classes


17

### Chargement du modèle EfficientNetB0 et adaptations du classifier à notre dataset

In [8]:
class EfficientNetB0_GTSDB(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = models.efficientnet_b0(weights="IMAGENET1K_V1")

        # Adapter la dernière couche
        in_features = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.model(x)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = EfficientNetB0_GTSDB(num_classes).to(device)


### Optimiseur et Loss

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

### Entraînement du modèle

In [ ]:
EPOCHS = 12

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        imgs, labels = imgs.to(device), labels.to(device)

        preds = model(imgs)
        loss = criterion(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    # Validation
    model.eval()
    val_loss = 0
    correct = 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            preds = model(imgs)
            val_loss += criterion(preds, labels).item()
            correct += (preds.argmax(1) == labels).sum().item()

    val_loss /= len(val_loader)
    val_acc = correct / len(val_data)

    print(f"Epoch {epoch+1} | TrainLoss={train_loss:.4f} | "
          f"ValLoss={val_loss:.4f} | ValAcc={val_acc:.4f}")
torch.save(model.state_dict(), os.path.join("partie_4", f"efficientnet_epoch{epoch+1}.pt"))



Epoch 1/1: 100%|██████████| 15/15 [00:23<00:00,  1.59s/it]


Epoch 1 | TrainLoss=0.0846 | ValLoss=0.2982 | ValAcc=0.8889


### Evaluation du modèle 

In [11]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        preds = model(imgs)

        correct += (preds.argmax(1) == labels).sum().item()
        total += labels.size(0)

print("Test Accuracy =", correct / total)


Test Accuracy = 0.9797979797979798
